In [1]:

import ccxt
import config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime
import time
import pytz
import json

# import warnings
# warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, './strategy')
from importlib import reload  

In [2]:
exchange = ccxt.kucoin({
    "apiKey": config.KUCOIN_API_KEY,
    "secret": config.KUCOIN_SECRET_KEY,
    "password": config.KUCOIN_API_PASSPHRASE
})

In [10]:
symbol = 'DOT/USDT'
# type = 'limit'  # or 'market', other types aren't unified yet
type = 'limit'
side = 'sell'
amount = 0.1  # your amount
price = 21.99  # your price
# overrides
params = {
    'stopPrice': 22.0,  # your stop price
    # 'type': 'stop',
    'stop': 'loss'
}
order = exchange.create_order(symbol, type, side, amount, price, params)
order

{'id': 'vs8f2o8us67chiue003bak53',
 'clientOrderId': '2e99abe8-103c-4a81-bbb6-80c45b8b09e1',
 'info': {'orderId': 'vs8f2o8us67chiue003bak53'},
 'timestamp': 1629413774968,
 'datetime': '2021-08-19T22:56:14.968Z',
 'lastTradeTimestamp': None,
 'symbol': 'DOT/USDT',
 'type': 'market',
 'side': 'sell',
 'price': 21.99,
 'amount': 0.1,
 'cost': None,
 'average': None,
 'filled': None,
 'remaining': None,
 'status': None,
 'fee': None,
 'trades': None}

In [3]:
order = exchange.create_market_sell_order('DOT/USDT', 0.01)

In [4]:
order

{'id': '611e84a785ce7500066a4378',
 'clientOrderId': 'fe8178cb-22dd-4afd-895c-0024f5adda13',
 'info': {'orderId': '611e84a785ce7500066a4378'},
 'timestamp': 1629389991218,
 'datetime': '2021-08-19T16:19:51.218Z',
 'lastTradeTimestamp': None,
 'symbol': 'DOT/USDT',
 'type': 'market',
 'side': 'sell',
 'price': None,
 'amount': 0.01,
 'cost': None,
 'average': None,
 'filled': None,
 'remaining': None,
 'status': None,
 'fee': None,
 'trades': None}

In [41]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string', 'strategy': 'string'})
ondf = odf[odf.orderId.isnull()]
# pd.isna(odf['orderId'])
ondf

,date_entered,exchange,pair,strategy,order_params,orderId
id,,,,,,


In [37]:
odf


,date_entered,exchange,pair,status,strategy,order_params,orderId
id,,,,,,,
10,2021-08-19 14:45:04.560470,KUCOIN,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25.50, ""qty"": 0.01}",611e86cc2b17a90006105932,<NA>
11,2021-08-19 14:45:05.560470,KUCOIN,BTC/USDT,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}",NaN,<NA>
2,2021-08-19 14:45:06.560470,KUCOIN,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25, ""qty"": 0.01}",NaN,<NA>
3,2021-08-19 14:45:07.560470,KUCOIN,BTC/USDT,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}",NaN,<NA>


In [3]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string'})
ondf = odf[odf.status=='new']
for index, orow in ondf.iterrows():
    module = __import__(orow.strategy)
    reload(module)

In [4]:

def run_bot():
    print(f'{datetime.now()} Checking for new orders...')
    odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string'})
    ondf = odf[odf.orderId.isnull()]
    pairs = ondf.pair.values
    tickers = exchange.fetchTickers (pairs)

    for oind, orow in ondf.iterrows():
        # print("index", index)
        module = __import__(orow.strategy)
        order_class = getattr(module, orow.strategy)
        order_params = json.loads(orow.order_params)
        order_instance = order_class(exchange, orow.pair, tickers[orow.pair], order_params)
        order = order_instance.execute()
        if order:
            odf.loc[oind, "orderId"] = order['id']
            odf.to_csv('order-data/orders.csv', index=True)
    # odf


schedule.every(10).seconds.do(run_bot)


while True:
    schedule.run_pending()
    time.sleep(1)

2021-08-19 17:53:19.126421 Checking for new orders...
2021-08-19 17:53:24.469401 Checking for new orders...
2021-08-19 17:53:29.785632 Checking for new orders...
2021-08-19 17:53:30.091620 Checking for new orders...
2021-08-19 17:53:35.708445 Checking for new orders...
2021-08-19 17:53:41.050638 Checking for new orders...
2021-08-19 17:53:41.614368 Checking for new orders...
2021-08-19 17:53:46.933644 Checking for new orders...
2021-08-19 17:53:52.303245 Checking for new orders...
BuyLevelOrder triggered for pair DOT/USDT with order_params {"trigger_price": 25.26, "qty": 0.02}
2021-08-19 17:53:53.393274 Checking for new orders...
2021-08-19 17:53:58.720870 Checking for new orders...
2021-08-19 17:54:03.434074 Checking for new orders...
2021-08-19 17:54:04.774204 Checking for new orders...
2021-08-19 17:54:10.097959 Checking for new orders...
2021-08-19 17:54:14.474295 Checking for new orders...
2021-08-19 17:54:15.792452 Checking for new orders...
2021-08-19 17:54:21.129264 Checking fo

KeyboardInterrupt: 

In [7]:
run_bot()

index 2021-08-19 14:45:04.560470
index 2021-08-19 14:45:04.560470


In [38]:
orow = ondf.iloc[0]
odf.index[odf.id == orow.id][0]

0

AttributeError: 'Series' object has no attribute 'id'

In [11]:
order_instance.execute()

order params: {'trigger_price': 25, 'qty': 0.01}


In [17]:
ondf

,exchange,pair,status,strategy,order_params
date,,,,,
2021-08-19 14:45:04.560470,KUCOIN,DOT/USDT,new,BuyLevelOrder,"{""trigger_price"": 25, ""qty"": 0.01}"
2021-08-19 14:45:04.560470,KUCOIN,BTC/USDT,new,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}"


In [44]:
odf

,exchange,pair,status,strategy,params
date,,,,,
2021-08-19 14:45:04.560470,KUCOIN,DOT/USDT,new,BuyLevel,{}


In [20]:
time.localtime()

time.struct_time(tm_year=2021, tm_mon=8, tm_mday=19, tm_hour=14, tm_min=42, tm_sec=3, tm_wday=3, tm_yday=231, tm_isdst=1)